In [23]:
import pandas as pd
import numpy as np
from sklearn.tree import DecisionTreeRegressor
from sklearn.ensemble import RandomForestRegressor

from sklearn.metrics import mean_absolute_error
from sklearn.model_selection import cross_val_score

import eli5
from eli5.sklearn import PermutationImportance

from ast import literal_eval
import re

In [4]:
df = pd.read_csv('data/men_shoes.csv', low_memory=False)

In [12]:
def run_model(feats, model=DecisionTreeRegressor(max_depth=5)):
    X = df[feats].values
    y = df['prices_amountmin'].values

    scores = cross_val_score(model, X, y, scoring='neg_mean_absolute_error')
    return np.mean(scores), np.std(scores)

In [16]:
df['brand_cat'] = df['brand'].map(lambda x: str(x).lower()).factorize()[0]

In [17]:
run_model(['brand_cat'])

(-58.133398968282776, 4.206122611474276)

In [18]:
run_model(['brand_cat'], model=RandomForestRegressor(max_depth=5, n_estimators=100))

(-57.32394965205956, 4.124301774688229)

In [21]:
df['features'].head()

0    [{"key":"Gender","value":["Men"]},{"key":"Shoe...
1    [{"key":"Gender","value":["Men"]},{"key":"Shoe...
2    [{"key":"Gender","value":["Men"]},{"key":"Colo...
3    [{"key":"Gender","value":["Men"]},{"key":"Colo...
4    [{"key":"Gender","value":["Men"]},{"key":"Colo...
Name: features, dtype: object

In [51]:
def parse_features(x):
    output_dict = {}
    if str(x) == 'nan': return {}
    
    features = literal_eval(x.replace('\\"', '"'))
    for item in features:
        key = item['key'].lower().strip()
        value = item['value'][0].lower().strip()
        output_dict[key] = value
        
    return output_dict

In [54]:
df['features_parsed'] = df['features'].map(parse_features)

In [62]:
df['features_parsed'].head(1).values

array([{'gender': 'men', 'shoe size': 'm', 'shoe category': "men's shoes", 'color': 'multicolor', 'manufacturer part number': '8190-w-navy-7.5', 'brand': 'josmo'}],
      dtype=object)

In [63]:
keys = set()

df['features_parsed'].map(lambda x: keys.update(x.keys()))

len(keys)

476

In [79]:
df['features_parsed'][0].keys()

dict_keys(['gender', 'shoe size', 'shoe category', 'color', 'manufacturer part number', 'brand'])

In [87]:
def get_feat_name(key):
    return 'feat_' + key

for key in keys:    
    df[get_feat_name(key)] = df['features_parsed'].map(lambda feats: feats[key] if key in feats.keys() else None)

In [88]:
df.columns

Index(['id', 'asins', 'brand', 'categories', 'colors', 'count', 'dateadded',
       'dateupdated', 'descriptions', 'dimension',
       ...
       'feat_heel height', 'feat_stylenumber',
       'feat_country////region of manufacture', 'feat_wheel type',
       'feat_lens material', 'feat_inv', 'feat_number of pieces',
       'feat_manufacturer', 'feat_age gender group', 'feat_bridge size:'],
      dtype='object', length=526)

In [100]:
keys_stat = {}
for key in keys:
    keys_stat[key] = df[~df[get_feat_name(key)].isnull()].shape[0] / df.shape[0] * 100

In [128]:
keys_stat['material']

34.9070021881838

In [107]:
{k : v for k, v in keys_stat.items() if v > 35}

{'color': 47.784463894967175,
 'manufacturer part number': 36.252735229759296,
 'gender': 50.17505470459519,
 'brand': 48.62691466083151}

In [130]:
# df['feat_brand_cat'] = df['feat_brand'].factorize()[0]
# df['feat_manufacturer part number_cat'] = df['feat_manufacturer part number'].factorize()[0]
# df['feat_gender_cat'] = df['feat_gender'].factorize()[0]
# df['feat_color_cat'] = df['feat_color'].factorize()[0]

# df['feat_sport_cat'] = df['feat_sport'].factorize()[0]

for key in keys:
    df[get_feat_name(key) + '_cat'] = df[get_feat_name(key)].factorize()[0]

In [115]:
df['brand'] = df['brand'].map(lambda x: str(x).lower())

In [118]:
df[df['brand'] == df['feat_brand']][['brand', 'feat_brand']].shape

(8846, 2)

In [187]:
# feats = ['brand_cat', 'feat_brand_cat', 'feat_manufacturer part number_cat', 'feat_gender_cat', 'feat_color_cat', 'feat_sport_cat']
feats = [col for col in df.columns if re.search('\w+_cat$', col)]

In [188]:
model = RandomForestRegressor(max_depth=5, n_estimators=100)
run_model(feats, model)

(-57.60240552501542, 4.236353819351158)

In [189]:
X = df[feats].values
y = df['prices_amountmin'].values

m = RandomForestRegressor(max_depth=5, n_estimators=100)
m.fit(X,y)

perm = PermutationImportance(m, random_state=1).fit(X,y)
eli5.show_weights(perm, feature_names=feats)

Weight,Feature
0.2479 ± 0.0124,brand_cat
0.1078 ± 0.0136,feat_material_cat
0.0138 ± 0.0012,feat_weight_cat
0.0118 ± 0.0019,feat_brand_cat
0.0088 ± 0.0005,feat_fabric content_cat
0.0071 ± 0.0026,feat_resizable_cat
0.0068 ± 0.0025,feat_adjustable_cat
0.0058 ± 0.0003,feat_shoe category_cat
0.0040 ± 0.0004,feat_fabric material_cat
0.0035 ± 0.0008,feat_color_cat
